# Suspicious Email Analysis, Sample From Hybrid-Analysis

In [33]:
# Import extract_msg, read in file and analyze headers
import extract_msg

suspect_mail_file = 'PO_18_12.msg'
msg = extract_msg.Message(suspect_mail_file)
msg_hdr = str(msg.header)
print(msg_hdr)

Return-Path: <info@ricongroup.com>
Received: from ricongroup.com ([188.209.52.201]) by mx.kundenserver.de
 (mxeue109 [217.72.192.67]) with ESMTP (Nemesis) id 1MPHNj-1iKh7a2upp-00PmVo
 for <contact@vins-tso.com>; Thu, 19 Dec 2019 17:27:11 +0100
From: info@ricongroup.com
To: contact@vins-tso.com
Subject: PO 18-12-2019
Date: 19 Dec 2019 08:27:12 -0800
Message-ID: <20191219082712.DA4A76788FE4A930@ricongroup.com>
MIME-Version: 1.0
Content-Type: multipart/mixed;
	boundary="----=_NextPart_000_0012_C2B69DE6.AAD179D4"
Envelope-To: <contact@vins-tso.com>
X-Spam-Flag: NO
X-UI-Filterresults: notjunk:1;V03:K0:OKLB9hp3O8I=:NGfMT4O04rMWYpmmJzeEKyZp8h
 GC9BO+IQwm0yvILevLt34WambE3brpO3PvD/l/0TXhG3Z7eN8AOwwdBGvly1USxk0BNcLT1id
 2LQocUXCrkUxNEdXk45K6l+jlPbhxDHfYo6UO0JPgyjJW0SqxO82DQxItinjJnlg2f5G6zzqd
 jPCZ5xLBSonAsam3PdSNc/5Rs5pV1hBR27+o8s3C66EhxG3ZUJ3D858qL46Vg+vKeCl8RjAjw
 PK3Nhl5niq3klkZTwA4e0yY0YzJ1tgXI4UOpADcJQ/jnk7g6AyBJpHIrlQGZXvNi9O1gpD6tB
 tgJjEE5aVqdwgmgKZj3TlA+bfgWVBeTDuBQiLA4di6zr9BeAM7JHLeS

In [50]:
# Extract IPv4 addresses and domain names from header
import re

ips_in_hdr = re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", msg_hdr)
urls_in_hdr = re.findall("[^@]+@[^\.]+\..+", msg_hdr)

print(ips_in_hdr)
print("\n")
print(urls_in_hdr)

['188.209.52.201', '217.72.192.67']


['Return-Path: <info@ricongroup.com>', '\nReceived: from ricongroup.com ([188.209.52.201]) by mx.kundenserver.de\n (mxeue109 [217.72.192.67]) with ESMTP (Nemesis) id 1MPHNj-1iKh7a2upp-00PmVo\n for <contact@vins-tso.com>; Thu, 19 Dec 2019 17:27:11 +0100', '\nFrom: info@ricongroup.com', '\nTo: contact@vins-tso.com', '\nSubject: PO 18-12-2019\nDate: 19 Dec 2019 08:27:12 -0800\nMessage-ID: <20191219082712.DA4A76788FE4A930@ricongroup.com>', '\nMIME-Version: 1.0\nContent-Type: multipart/mixed;\n\tboundary="----=_NextPart_000_0012_C2B69DE6.AAD179D4"\nEnvelope-To: <contact@vins-tso.com>']


In [3]:
# View body of email for any URL's, see above to extract 
print(msg.body)

Dear Sir,      

     Attached is a copy of  PO, kindly check and approve by sending
back PI. 
NOTE: Please, include your full address detail in the PI as we prepare
to visit your company next year for a better business relationship 


Thanks,
 


Thanks & Regards,
Richa
(marketing manager)
Ricongroup Pvt. LTD
618, Ansal Chamber - II, 6, Bhikaji Cama Place, New Delhi - 110066
Ph. -　91-11-26190209　 / 26109031 / 26107961 Mobile :　(91)-9910455279

info@ricongroup.com/ <mailto:info@ricongroup.com/>
marketing@ricongroup.com




In [51]:
# Use attachment class to extract any attachments the email may contain 
# !!!WARNING!!! This will save file to current dir, ensure you are in a safe environment

for msg in msg.attachments:
    print(msg.save())

PO18-12-2019.xlsx


> We have a few different options at this point.  We can write Python code to get the hash of the file,
> or get it via the command line. Additionally, we could forego using Python to submit the file since it has
> been saved to our folder, and navigate to our favorite sandbox for submission.

In [61]:
# Submit file to VT (or your favorite sandbox)
# Don't forget about IPv4 addresses and domains from earlier
import requests

try:
    url_req = "https://www.virustotal.com/vtapi/v2/file/scan"

    vt_key =  'your_api_key_here'

    files = {'file': ('PO18-12-2019.xlsx', open('PO18-12-2019.xlsx', 'rb'))}

    url_resp = requests.post(url_req, files, params=params)

except requests.ConnectionError as e:
    print(e)
    

print(url_resp.json())


{'scan_id': '38c45906d9141935071b4775e64d36652d20fea66c4252736191b5979ad9d2e4-1577196494', 'sha1': '0f50038bd2e98eada891be455b386f96d9cd81e1', 'resource': '38c45906d9141935071b4775e64d36652d20fea66c4252736191b5979ad9d2e4', 'response_code': 1, 'sha256': '38c45906d9141935071b4775e64d36652d20fea66c4252736191b5979ad9d2e4', 'permalink': 'https://www.virustotal.com/file/38c45906d9141935071b4775e64d36652d20fea66c4252736191b5979ad9d2e4/analysis/1577196494/', 'md5': 'd8b7b554a7c5bf9ab5e4ed99bbb4bda7', 'verbose_msg': 'Scan request successfully queued, come back later for the report'}


In [74]:
# Submit to Hybrid-Analysis
import requests


try:
    url = 'https://www.hybrid-analysis.com/api/v2/search/hash' 

    headers = {'User-Agent': 'Falcon Sandbox', 'api-key': 'your_api_key_here'}

    payloads = {'hash': '52a664b81345bbd4b04ae070e85f1d48389969a9'}  

    response = requests.post(url, headers=headers, data=payloads) 
    
except requests.ConnectionError as e:
    print(e)

print(response.json())


[{'job_id': '5dfbaa4d217d93849ff262e9', 'environment_id': 100, 'environment_description': 'Windows 7 32 bit', 'size': 17272, 'type': 'Microsoft Excel 2007+', 'type_short': ['xlsx', 'office'], 'target_url': None, 'state': 'SUCCESS', 'error_type': None, 'error_origin': None, 'submit_name': 'c77febcf9342fd3adf06eee2265341a427f710486759e66c7c24cccb63c08c38_1576774219866_PO18-12-2019.xlsx', 'md5': '2ecb65782c935d245687366e2148e892', 'sha1': '52a664b81345bbd4b04ae070e85f1d48389969a9', 'sha256': 'c77febcf9342fd3adf06eee2265341a427f710486759e66c7c24cccb63c08c38', 'sha512': 'c3329e420380da531677fad462513e756a5d04966415e01eeeec9fe91626bd4d7008f8ad4304f960ed5bd643728c7182c6e69381c45fd26032b53e1c87666d74', 'ssdeep': '384:wqiCe8qmj9KhKVxbo6IpbElfXmeJxQQHwFe4ubTRkLJiSY1:wqiCxqmhtVxE6ib4hfp2e4u/RJSY1', 'imphash': 'Unknown', 'av_detect': 54, 'vx_family': 'CVE-2017-11882', 'url_analysis': False, 'analysis_start_time': '2019-12-19T16:50:21+00:00', 'threat_score': 88, 'interesting': False, 'threat_level'